![Neptune + skorch](https://neptune.ai/wp-content/uploads/2023/09/skorch.svg)

# Neptune + skorch

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/skorch/notebooks/Neptune_Skorch.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/skorch/notebooks/Neptune_Skorch.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/skorch-integration/e/SKOR-32"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/skorch/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

In this guide, you will learn how to use `NeptuneLogger()` to log training metrics to Neptune.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
! pip install neptune torch scikit-learn skorch matplotlib

## Import libraries

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import neptune
from neptune.types import File
from skorch.callbacks import NeptuneLogger, Checkpoint
from skorch import NeuralNetClassifier

## Loading data
Use scikit-learn's ```fetch_openml``` to load MNIST data.

In [ ]:
mnist = fetch_openml("mnist_784", as_frame=False, cache=False)

## Preprocessing data

Each image of the MNIST dataset is encoded in a 784 dimensional vector, representing a 28 x 28 pixel image. Each pixel has a value between 0 and 255, corresponding to the grey-value of a pixel.<br />
The above ```fetch_mldata``` method returns ```data``` and ```target``` as ```uint8``` which we convert to ```float32``` and ```int64``` respectively.

In [ ]:
X = mnist.data.astype("float32")
y = mnist.target.astype("int64")

To avoid big weights that deal with the pixel values in the range [0, 255], we scale `X` down. A commonly used range is [0, 1].

In [ ]:
X /= 255.0

In [ ]:
X.min(), X.max()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
assert X_train.shape[0] + X_test.shape[0] == mnist.data.shape[0]

In [ ]:
X_train.shape, y_train.shape

### Print a selection of training images and their labels

In [ ]:
def plot_example(X, y):
    """Plot the first 5 images and their labels in a row."""
    for i, (img, y) in enumerate(zip(X[:5].reshape(5, 28, 28), y[:5])):
        plt.subplot(151 + i)
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
        plt.title(y)

In [ ]:
plot_example(X_train, y_train)

## Build a neural network with PyTorch
Next we'll build a simple, fully connected neural network with one hidden layer. The input layer has 784 dimensions (28 x 28), the hidden layer has 98 (= 784 / 8), and the output layer 10 neurons, representing digits 0 - 9.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
mnist_dim = X.shape[1]
hidden_dim = int(mnist_dim / 8)
output_dim = len(np.unique(mnist.target))

In [ ]:
mnist_dim, hidden_dim, output_dim

A neural network in PyTorch's framework.

In [ ]:
class ClassifierModule(nn.Module):
    def __init__(
        self,
        input_dim=mnist_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim,
        dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.dropout = nn.Dropout(dropout)

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, X, **kwargs):
        X = F.relu(self.hidden(X))
        X = self.dropout(X)
        X = F.softmax(self.output(X), dim=-1)
        return X

## Start a run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in the public project [common/skorch-integration](https://app.neptune.ai/common/skorch-integration). **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    name="skorch-example",
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
run = neptune.init_run(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project="common/skorch-integration",
    name="skorch-example",
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to to initialize the `NeptuneLogger`. You'll see the metadata appear in the app.

## Create NeptuneLogger 

In [ ]:
neptune_logger = NeptuneLogger(run, close_after_train=False)

In [ ]:
checkpoint_dirname = "./checkpoints"
checkpoint = Checkpoint(dirname=checkpoint_dirname)

## Initialize a trainer and pass neptune_logger

In [ ]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    device=device,
    callbacks=[neptune_logger, checkpoint],
)

In [ ]:
net.fit(X_train, y_train);

## More options

### Log model weights
Use the Checkpoint Callback to save the model files to disk. You can then upload the files to Neptune.

In [ ]:
neptune_logger.run["training/model/checkpoints"].upload_files(checkpoint_dirname)

### Log test score

In [ ]:
y_pred = net.predict(X_test)
neptune_logger.run["training/test/acc"] = accuracy_score(y_test, y_pred)

### Log misclassified images

In [ ]:
error_mask = y_pred != y_test
plot_example(X_test[error_mask], y_pred[error_mask])

for x, y_hat, y in zip(X_test[error_mask], y_pred[error_mask], y_test[error_mask]):
    x_reshaped = x.reshape(28, 28)
    neptune_logger.run["training/test/misclassified_images"].append(
        File.as_image(x_reshaped), description=f"y_pred={y_hat}, y_true={y}"
    )

## Stop logging
Once you are done logging, stop tracking the run.

In [ ]:
neptune_logger.run.stop()

## Explore the results in Neptune

Follow the link to the run and explore metadata (such as metrics, hyperparameters, and model checkpoints) that were logged to the run in Neptune.

You can also check out an [example run](https://app.neptune.ai/o/common/org/skorch-integration/e/SKOR-32/dashboard/skorch-dashboard-97de6fa9-92dd-4b76-9842-b1fbe9cc992e).